In [1]:
#Instalação das Libraries
!pip install yfinance==0.2.41
!pip install crewai==0.28.8
!pip install 'crewai[tools]'
!pip install langchain==0.1.20
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.0.38
!pip install duckduckgo-search==5.3.0
%pip install --upgrade --quiet duckduckgo-search

  Using cached duckduckgo_search-5.3.0-py3-none-any.whl.metadata (18 kB)
Using cached duckduckgo_search-5.3.0-py3-none-any.whl (21 kB)
  Attempting uninstall: duckduckgo-search
    Found existing installation: duckduckgo_search 6.2.10
    Uninstalling duckduckgo_search-6.2.10:
      Successfully uninstalled duckduckgo_search-6.2.10
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

In [3]:
# Aqui usando um dos métodos da biblioteca onde passamos o código da Ação, a data início e a data fim E fazemos download desses dados

# Criando a Tool que será utilizada pelo Agente
def fetch_preco_stock(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock 

yahoo_finance_tool = Tool(
     name = "Yahoo Finance Tool",
     description = "Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
     func = lambda ticket: fetch_preco_stock(ticket)
)

In [4]:
# IMPORTANDO OPENAI LLM - GPT
os.environ['OPENAI_API_KEY'] = "MINHA_KEY"
llm = ChatOpenAI(model = "gpt-3.5-turbo")

In [5]:
stockPriceAnalyst = Agent(
    role = "Senior Stock Price Analyst",
    goal = "Find the {ticket} stocks price and analyses trends",
    backstory = """ You're a highly experienced in analyzing the price of an specific stock
    and make predictions about its future price.""",
    verbose = True,
    llm = llm,
    max_iter = 5, # Ou seja pode ficar no ciclo de Perception, Reason Action até 5 vezes
    memory = True,
    tools = [yahoo_finance_tool],
    allow_delegation = False
)

In [6]:
getStockPrice = Task(
    description = "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output = """ Specify the current trend stock price - up, down or sideways.
    eg. stock = 'APPL, price UP'
    """,
    agent = stockPriceAnalyst
)

In [7]:
# IMPORTAR TOOL DE PESQUISA
search_tool = DuckDuckGoSearchResults(backend = 'news', num_results = 10)

In [8]:
newsAnalyst = Agent(
    role = "Stock News Analyst",
    goal = """Create a short summary of the market news related to the stock {ticket} company. Specify the current 
    trend - up, down or sideways with the news context. For each request stock asset, specify a number 
    between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory = """ You're a highly experienced in analyzing the market trends and news and have tracked assets for
    more than 10 years. 

    You're also master level analyst in the tradicional markets and have deep understanding of human psychology.

    You understand news, theirs titles and information, but you look at those with a health dose of skepticism.
    You consider also the source of the news articles.
""",
    verbose = True,
    llm = llm,
    max_iter = 10,
    memory = True,
    tools = [search_tool],
    allow_delegation = False
)

In [9]:
get_news = Task (
    description = f"""Take the stock and always include BTC to it (if not request). 
    Use the search tool to search each one individually.
    
    The current date is {datetime.now()}.
    
    Compose the results into a helpfull report""",
    expected_output = """A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
    """,
    agent = newsAnalyst
)

In [10]:
stockAnalystWrite = Agent(
    role = "Senior Stock Analyst Writer",
    goal = """Analyze the trends price and news and write an insightful compelling and informative 3 paragraph long newsletter based on the stock report
    and price trend.""",
    backstory = """You're widely accepted as the best stock analyst in the market. You understand complex concepts
    and create compelling stories and narratives that resonate with wider audiences.

    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses.
    You're able to hold multiple opinions when analyzing anything.
""",
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    allow_delegation = True
)

In [11]:
writeAnalyses = Task (
    description = """Use the stock price trend and the stock news report to create ana analyses and write the
    newsletter about the {ticket} company that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary.
    """,
    expected_output = """An eloquent 3 paragraphs newsletter formatted as markdown in an easy readable manner.
    It should contain:
    - 3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fear/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways.
    """,
    agent = stockAnalystWrite,
    context = [getStockPrice, get_news]
)

In [12]:
# Utilizado o Hierárquico, pois no fim do dia o Agente Que escreve será o Administrador dos outros agentes

crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
    tasks = [getStockPrice, get_news, writeAnalyses],
    verbose = 2,
    process= Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15
)

In [13]:
results = crew.kickoff(inputs = { 'ticket': 'AAPL' })

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to delegate the task of analyzing the stock AAPL price history and creating a trend analysis to the Senior Stock Price Analyst. They have the expertise to handle this task effectively.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Stock Price Analyst", "task": "Analyzing AAPL stock price history and creating trend analysis", "context": "The task involves analyzing the historical price data of AAPL stock and determining whether the current trend is up, down, or sideways. Please use all available resources and tools to provide an accurate analysis."}


> Entering new CrewAgentExecutor chain...
I need to gather historical price data for AAPL stock from the last year in order to analyze trends accurately.

Action: Yahoo Finance Tool
Action Input: {"ticket": "AAPL"}

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643372   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043747   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884872   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546371   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date      

[*********************100%%**********************]  1 of 1 completed

I need to use the Yahoo Finance Tool to fetch the stock price information for AAPL and BTC.

Action: Yahoo Finance Tool
Action Input: {"ticker": "AAPL"} 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643372   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043747   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884872   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546371   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990

Final Answer: 
The current stock price of AAPL is $209.82 as of August 7, 2024.

> Finished chain.
 

The current stock price of AAPL is $209.82 as of August 7, 2024.

Thought: I have gathered the stock price information for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Stock News Analyst", "question": "What is the latest news affecting AAPL?", "context": "We need to include a summary based on news in our report."}

> Entering new CrewAgentExecutor chain...
I should use the duckduckgo_results_json tool to search for the latest news affecting AAPL and analyze the information with a critical eye.

Action: duckduckgo_results_json
Action Input: {"query": "latest news AAPL stock"} 

[snippet: Apple (AAPL), like most stocks, has been on volatile ride lately bouncing up and down between 195 and 240. However, the options market could be indicating that the stock might settle around the 215 mark over the next few weeks., title: Bulls vs. Bears: AAPL Max Pain Points to a P

In [18]:
list(results.keys())

['final_output', 'tasks_outputs']

In [15]:
results['final_output']

"# AAPL Stock Newsletter\n\n### Executive Summary:\n- AAPL stock is currently experiencing volatility with mixed sentiments from investors.\n- Recent news includes Warren Buffett's stake reduction and insider sales within the company.\n- Fear/greed score remains neutral, indicating a balanced sentiment in the market.\n\n### Introduction:\nThe latest developments surrounding AAPL stock have stirred the interest of investors, with a blend of bullish and bearish sentiments shaping the current landscape. Uncertainty looms as Warren Buffett's recent move to reduce his stake in the company has sparked questions among stakeholders. Insider sales further contribute to the fluctuating nature of AAPL stock.\n\n### Main Analysis:\nThe options market hints at a potential settlement around $215 in the near future, adding an element of anticipation to the stock's trajectory. Despite these challenges, analysts maintain a positive outlook with a Buy rating and a price target of $256, supported by reas

In [16]:
len(results['tasks_outputs'])

3

In [17]:
Markdown(results['final_output'])

# AAPL Stock Newsletter

### Executive Summary:
- AAPL stock is currently experiencing volatility with mixed sentiments from investors.
- Recent news includes Warren Buffett's stake reduction and insider sales within the company.
- Fear/greed score remains neutral, indicating a balanced sentiment in the market.

### Introduction:
The latest developments surrounding AAPL stock have stirred the interest of investors, with a blend of bullish and bearish sentiments shaping the current landscape. Uncertainty looms as Warren Buffett's recent move to reduce his stake in the company has sparked questions among stakeholders. Insider sales further contribute to the fluctuating nature of AAPL stock.

### Main Analysis:
The options market hints at a potential settlement around $215 in the near future, adding an element of anticipation to the stock's trajectory. Despite these challenges, analysts maintain a positive outlook with a Buy rating and a price target of $256, supported by reassurances related to new EU regulations. The stock's resilience in the face of adversity, outperforming the S&P 500 in the past, continues to be a point of interest for investors.

### Summary and Future Trend Prediction:
Considering the current trend of mixed sentiments and recent events, AAPL stock may witness further volatility in the short term. While uncertainties persist, the overall sentiment remains cautiously optimistic, leaning towards a bearish tilt due to recent developments. As investors navigate through these turbulent times, keeping a close eye on the evolving situation will be crucial in determining the stock's future direction.